In [ ]:
import os
HOME = os.environ["HOME"]
os.environ["CARDIAC_GWAS_REPO"] = CARDIAC_GWAS_REPO = f"{HOME}/01_repos/CardiacGWAS"
os.environ["CARDIAC_COMA_REPO"] = CARDIAC_COMA_REPO = f"{HOME}/01_repos/CardiacCOMA/"
os.environ["GWAS_REPO"] = GWAS_REPO = f"{HOME}/01_repos/GWAS_pipeline/"

MLRUNS_DIR = f"{CARDIAC_COMA_REPO}/mlruns"
#os.chdir(CARDIAC_COMA_REPO)

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

import os, sys

import torch
import torch.nn.functional as F

from CardiacCOMA.config.cli_args import overwrite_config_items
from CardiacCOMA.config.load_config import load_yaml_config, to_dict
from CardiacCOMA.utils.helpers import get_datamodule, get_lightning_module
from CardiacCOMA.utils.mlflow_helpers import get_model_pretrained_weights
from CardiacCOMA.utils.CardioMesh.CardiacMesh import transform_mesh

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image, display, Markdown, clear_output

import pandas as pd
import shlex
from subprocess import check_output

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from IPython import embed
sys.path.insert(0, '..')

from copy import deepcopy
from pprint import pprint

from typing import List
from tqdm import tqdm

import pandas as pd

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual

from auxiliary import load_data
from auxiliary import get_model_pretrained_weights
import warnings

from image_helpers import merge_pngs

In [ ]:
def load_data(b):
    global meshes, procrustes_transforms
    print("Loading mesh data...")
    meshes = pkl.load(open(f"{CARDIAC_COMA_REPO}/data/cardio/LV_meshes_at_ED_35k.pkl", "rb"))
    print("Mesh data loaded successfully.")
    
    print("Loading Procrustes transforms...")
    procrustes_transforms = pkl.load(open(f"{CARDIAC_COMA_REPO}/data/cardio/procrustes_transforms_35k.pkl", "rb"))
    print("Procrustes transform loaded successfully.")
    
button = widgets.Button(
    description='Load data',
    disabled=False,
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

load_data(1)
#button.on_click(load_data)
#button    

In [ ]:
ids = [ str(id) for id in meshes ] 

In [ ]:
import random
pv.set_plot_theme("document")

faces, _ = pkl.load(open(f"{CARDIAC_COMA_REPO}/data/cardio/faces_and_downsampling_mtx_frac_0.1_LV.pkl", "rb")).values()
faces = np.c_[np.ones(faces.shape[0]) * 3, faces].astype(int)

color_palette = list(pv.colors.color_names.values())
random.shuffle(color_palette)

# Select genetic locus

In [ ]:
good_runs_df = pd.read_csv(f"{CARDIAC_GWAS_REPO}/results/good_runs.csv")
run_ids = good_runs_df.run_id.to_list()

gwas_harmonized_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}.tsv"
gwas_selected_snps_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}__selected_snps.tsv"
COMA_GWAS_SUMMARY = "results/gwas_loci_summary_across_runs.csv"
LOGP_PATH = "results/log10p_for_selected_snps_across_gwas.csv"

In [ ]:
good_runs_df = pd.read_csv(f"{CARDIAC_GWAS_REPO}/results/good_runs.csv")
run_ids = good_runs_df.run_id.to_list()
gwas_loci_summary_across_runs_df = pd.read_csv(COMA_GWAS_SUMMARY)

# get index of best locus/variable
idx = gwas_loci_summary_across_runs_df.groupby(["region"])["P"].transform(min) == gwas_loci_summary_across_runs_df["P"]

best_association_per_region = gwas_loci_summary_across_runs_df[idx].sort_values("region")
best_snps = set(best_association_per_region.SNP)

regions = { 
    f"{assoc[1].region} ({assoc[1].P:.1e})": assoc[1].region 
    for assoc in best_association_per_region.sort_values("P").iterrows() 
} 

del regions["chr6_79 (4.5e-20)"]

_best_association_per_region = best_association_per_region.set_index("region")
# assoc = _best_association_per_region.loc[region]
#run_id, z_variable = assoc.run, assoc.pheno[-4:]

In [ ]:
_best_association_per_region.head(5)

In [ ]:
N_STEPS = 100
STEP = 1 / N_STEPS
quantiles = np.arange(N_STEPS+1)/N_STEPS

quantile_ranges = [(quantiles[i], quantiles[i+1]) for i, q in enumerate(quantiles[:-1])]

In [ ]:
#aligned_meshes = { id:  transform_mesh(mesh,  **procrustes_transforms[id]) for id, mesh in meshes.items() }
#rmsd = {id: np.array([np.sqrt((p**2).sum()) for p in mesh]).mean() for id, mesh in aligned_meshes.items()}
#pkl.dump(aligned_meshes, open("lved_aligned_meshes.pkl", "wb"))
#pkl.dump(rmsd, open("lved_rmsd.pkl", "wb"))

In [ ]:
aligned_meshes = pkl.load(open("lved_aligned_meshes.pkl", "rb"))
rmsd = pkl.load(open("lved_rmsd.pkl", "rb"))

In [ ]:
# 
# scaled_meshes = []
# msd = []
#             
# for id in ids:
#     
#     mesh = transform_mesh(
#         meshes[id], 
#         **procrustes_transforms[id]
#     )
#         
#     # mean-squared deviation
#     msd.append(np.array([np.sqrt((p**2).sum()) for p in mesh]).mean())
#     scaled_meshes.append(mesh)
#     
# scaled_meshes = np.array(scaled_meshes)
# msd = np.array(msd)

In [ ]:
from typing import Sequence, Dict
from z_effect_on_shape import avg_shape_in_q_range, get_ids_in_range, plot_mesh

In [ ]:
# TEST
# ids_in_range = list(meshes.keys())[0:100]
# avg_shape = np.array([meshes[id] / rmsd[id] for id in ids_in_range]).mean(0)
# rmsd_in_range = np.array([rmsd[id] for id in ids_in_range]).mean()
# avg_shape *= rmsd_in_range
# avg_shape

In [ ]:
exp_id = '1'

regions = [
   "chr17_27",
   "chr12_69",
   "chr6_78",
   "chr2_108",
   "chr11_2",
   "chr5_103",
   "chr1_124",
   "chr6_84",
   "chr2_69",
   "chr12_67"
]

q_ranges = [(0.00, 0.01), (0.095, 0.105), (0.45, 0.55), (0.895, 0.905), (0.99, 1.0)]

In [ ]:
VERBOSE = False
WARNINGS = False 

if WARNINGS:
    warnings.filterwarnings('default')    
else:
    warnings.filterwarnings('ignore')

    
for run_id in tqdm(sorted(os.listdir(f"{CARDIAC_COMA_REPO}/mlruns/1"))):
    
    try:
        z_filepath = f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/latent_vector.csv"
        zs = pd.read_csv(z_filepath).set_index("ID").columns
    except:
        continue
        
    for z in zs:
                
        #assoc = _best_association_per_region.loc[region]
        #run_id, z = assoc.run, assoc.pheno[-4:]
    
        z_effect_figs_dir = f"{CARDIAC_COMA_REPO}/mlruns/1/{run_id}/artifacts/z_effect_on_shape"
        os.makedirs(z_effect_figs_dir, exist_ok=True)
        
        filenames = { (q0, q1): f"{z_effect_figs_dir}/{z}_{q0}-{q1}.png" for q0, q1 in q_ranges }
    
        for (q0, q1), filename in filenames.items():
              
            if os.path.exists(filename):
                if VERBOSE: print(f"File {filename} already exists.")
                continue                        
            
            avg_mesh = avg_shape_in_q_range(meshes=aligned_meshes, rmsd=rmsd, quantile_range=(q0, q1), exp_id=exp_id, run_id=run_id, z=z)  
            
            if np.isnan(avg_mesh).all(): break
                                
            ofilename = filenames[(q0, q1)]
            plot_mesh(avg_mesh, faces, ofilename)                                    
           
        try:            
            filename = f"{z_effect_figs_dir}/{z}.png"
            if not os.path.exists(filename):
                merge_pngs(filenames.values(), output_png=filename, how="horizontally")
        except:
            pass
        
    if VERBOSE:
        print(f"{run_id[:5]}")

In [ ]:
for region in tqdm(regions):
  
  assoc = _best_association_per_region.loc[region]
  run_id, z = assoc.run, assoc.pheno[-4:]

  df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/latent_vector.csv").set_index("ID")

  filenames = []

  for q0, q1 in q_ranges:  
    
    filename = f"results/figs/{region}_{q0}-{q1}.png"    
    filenames.append(filename)  
    
    if os.path.exists(filename):
        print(f"File {filename} already exists.")
        continue
    
    z_bounds = df.quantile([q0, q1])[z]                
        
    ids = list(df[ 
        (z_bounds[q0] < df[z]) & (df[z] < z_bounds[q1])
    ][z].index) 
      
    scaled_meshes = []
    msd = []
    
    for id in ids:
        mesh = transform_mesh(
            meshes[str(id)], 
            **procrustes_transforms[str(id)]
        )
        
        # mean-squared deviation
        msd.append(np.array([np.sqrt((p**2).sum()) for p in mesh]).mean())
        scaled_meshes.append(mesh)
    
    scaled_meshes = np.array(scaled_meshes)
    msd = np.array(msd)
            
    avg_mesh = np.array([scaled_meshes[i] / msd[i] for i, _ in enumerate(scaled_meshes)]).mean(0)
    avg_mesh = avg_mesh * msd.mean()
                
    pv.set_plot_theme("document")
    pl = pv.Plotter(off_screen=True, notebook=False)
    
    pl.camera.position = (300, 0.0, 0.0)
    pl.camera.azimuth = 95
    
    mesh = pv.PolyData(avg_mesh, faces)
    pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
    
    pl.screenshot(filename);
    
  
  merge_pngs(filenames, output_png=f"{region}.png", how="horizontally")

In [ ]:
exp_id = '1'
# region = "chr17_27"
region = "chr12_69"
assoc = _best_association_per_region.loc[region]
run_id, z = assoc.run, assoc.pheno[-4:]
filename = "latent_vector.csv"
df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/{filename}")
df = df.set_index("ID")

for q0, q1 in quantile_ranges:
  
    z_bounds = df.quantile([q0, q1])[z]                
    
    ids = list(df[ 
        (z_bounds[q0] < df[z]) & (df[z] < z_bounds[q1])
    ][z].index) 
    
    scaled_meshes = []
    msd = []
    
    for id in ids:
        mesh = transform_mesh(
            meshes[str(id)], 
            **procrustes_transforms[str(id)]
        )
        
        # mean-squared deviation
        msd.append(np.array([np.sqrt((p**2).sum()) for p in mesh]).mean())
        scaled_meshes.append(mesh)
    
    scaled_meshes = np.array(scaled_meshes)
    
    msd = np.array(msd)            
    avg_mesh = np.array([scaled_meshes[i] / msd[i] for i, _ in enumerate(scaled_meshes)]).mean(0)
    avg_mesh = avg_mesh * msd.mean()
    
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    mesh = pv.PolyData(avg_mesh, faces)
    pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
    
    clear_output()
    print()
    pl.show(interactive=True, interactive_update=True)

In [ ]:
exp_id = '1'
STEP = 0.01

region_w = widgets.Select(options=regions, description="Locus: \n",)
display(region_w)
region = region_w.value

# _best_association_per_region = best_association_per_region.set_index("region")
assoc = _best_association_per_region.loc[region]
run_id, z = assoc.run, assoc.pheno[-4:]

quantile_inf_w=widgets.FloatSlider(min=0.00, max=0.99, step=STEP)
display(quantile_inf_w)    

button = widgets.Button(description="Plot mesh")
out = widgets.Output()

def plot_mesh(b):
    
    filename = "latent_vector.csv"
    df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/{filename}")
    df = df.set_index("ID")
    
    # z_w = widgets.Select(options=[f"z{str(i).zfill(3)}" for i in range(df.shape[1])])    
    
    with out:

        quantile_range = quantile_inf_w.value
        z_bounds = df.quantile([quantile_range, STEP+quantile_range])[z]                
        
        ids = list(df[ 
            (z_bounds[quantile_range] < df[z]) & (df[z] < z_bounds[quantile_range+STEP])
        ][z].index)    
                
        # COMPUTE AVERAGE MESH                        
        avg_mesh = np.array([
            transform_mesh(
                meshes[str(id)], 
                **procrustes_transforms[str(id)]
            ) for id in ids
        ]).mean(axis=0)

        clear_output()        
        
        pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
        mesh = pv.PolyData(avg_mesh, faces)
        pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
        pl.show(interactive=True, interactive_update=True)
        
button.on_click(plot_mesh)
# displaying button and its output together
widgets.VBox([button,out])    

___

# Select `run_id` / `z` variable

In [ ]:
from PIL import Image

exp_id = '1'
STEP=0.01
run_id_w = widgets.Select(options=sorted(run_ids))
display(run_id_w)

filename = "latent_vector.csv"
df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/{filename}")
df = df.set_index("ID")

z_w = widgets.SelectionSlider(options=[f"z{str(i).zfill(3)}" for i in range(df.shape[1])])
display(z_w)

quantile_range_w=widgets.FloatSlider(min=0, max=0.99, step=STEP)
display(quantile_range_w)

button = widgets.Button(description="Plot mesh")

out = widgets.Output()


def plot_mesh(b):
    
    with out:

        z = z_w.value
        run_id = run_id_w.value
        quantile_range = quantile_range_w.value
        z_bounds = df.quantile([quantile_range_w.value, quantile_range_w.value+STEP])[z]                
        
        ids = list(df[ 
            (z_bounds[quantile_range_w.value] < df[z]) & (df[z] < z_bounds[quantile_range_w.value+STEP])
        ][z].index) 
        
        print(len(ids))
                
        clear_output()      
        print(z, run_id)
        manhattan_file = f"{CARDIAC_COMA_REPO}/mlruns/1/{run_id}/artifacts/GWAS_adj_10PCs/figures/GWAS__{z}__1_{run_id}__manhattan.png"
        qq_file = f"{CARDIAC_COMA_REPO}/mlruns/1/{run_id}/artifacts/GWAS_adj_10PCs/figures/GWAS__{z}__1_{run_id}__QQ-plot.png"
        
        # COMPUTE AVERAGE MESH                        
        avg_mesh = np.array([
            transform_mesh(
                meshes[str(id)], 
                **procrustes_transforms[str(id)]
            ) for id in ids
        ]).mean(axis=0)

                      
        pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
        mesh = pv.PolyData(avg_mesh, faces)
        pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
        pv.camera
        pl.show(interactive=True, interactive_update=True)
        
        display(Image.open(manhattan_file))
        # display(Image.open(qq_file))
        
button.on_click(plot_mesh)
# displaying button and its output together
widgets.VBox([button,out])        

In [ ]:
pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)